# Introduction
Kalman filtering is used for many applications including:
- filtering noisy signals
- generating non-observable states
- and predicting future states

**Filtering noisy signals** is essential since many sensors have an output that is noisy to be used directly, and Kalman filtering lets you account for the uncertainty in the signal/state.

Kalman filter is such a general and powerful tool for **combining information** in the presence of uncertainty. At times its ability to extract accurate information seems almost magical.

Another nice feature of the Kalman filter is that it can be used to predict future states. This is useful when you have large time delays in your sensor feedback as this can cause instability in a motor control system.


Kalman filters produce the optimal estimate for a linear system. As such, a sensor or system must have(or be close to) a linear response in order to apply a Kalman filter. Techniques for working with non-linear sytems will be discussed later.





# What is it?
You can use a Kalman filter in any place where you have **uncertain information** about some dynamic system, and you can make an **educated guess** about what the system is going to do next.

Even if messy reality comes along and interferes with the clean motion you guessed about, the Kalman filter will often do a very good job of figuring out what actually happened. It can take advantage of correlations between phenomena that you maybe wouldn't have thought to exploit!

Kalman filters are ideal for systems which are **continuously changing**. They have the advantage that they are light on memory (they don’t need to keep any history other than the previous state), and they are very fast, making them well suited for real time problems and embedded systems.


# What can we do with a Kalman filter?

Let's use a visual example: you've built a little robot that can wander around in the woods, and the robot needs to know exactly where it is so that it can naviagate.
![](http://www.bzarg.com/wp-content/uploads/2015/08/robot_forest-300x160.png)

*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*



We'll say that our robot has a state **$\overrightarrow{x_k}$**, which is just a position and a velocity:

**$$\overrightarrow{x_k}=(\overrightarrow{p}, \overrightarrow{v})$$**

Note that the state is just a In our example it’s position and velocity, but it could be data about the amount of fluid in a tank, the temperature of a car engine, the position of a user’s finger on a touchpad, or any number of things you need to keep track of. list of numbers underlying configuration of our system; it could be anything.

Our robot also has a GPS sensor, which is accurate to about 10 meters, which is good, but it needs to know its location more precisely than 10 meters. There are lots of gullies and cliffs in these woods, and if the robot is wrong by more than a few feet, it could fall off a cliff. So GPS by itself is not good enough.
![](http://www.bzarg.com/wp-content/uploads/2015/08/robot_ohnoes.png)

*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*



There are **two types of equations** for the Kalman Filter.
1. The first are the prediction equations: these equations predict what the current state is based on the previous state and the commanded action
2. The second set of equations known as the **update equations** look at 
  - your input sensors, 
  - how much you trust each sensor, and 
  - how much you trust your overall state estimate. This filter works by predicting the current state using the prediction equations followed by checking how good of a job it did predicting by using the update equations. This process is repeated continuously to update teh current state.

![](http://i0.wp.com/robotsforroboticists.com/wordpress/wp-content/uploads/2014/01/kalmanEquationFlow.png)


We might also know something about how the robot moves: It knows the commands sent to the wheel motors, and its knows that if it’s headed in one direction and nothing interferes, at the next instant it will likely be further along that same direction. But of course it doesn’t know everything about its motion: It might be buffeted by the wind, the wheels might slip a little bit, or roll over bumpy terrain; so the amount the wheels have turned might not exactly represent how far the robot has actually traveled, and the prediction won’t be perfect.

The **GPS sensor** tells us something about the state, but only indirectly, and with some uncertainty or inaccuracy. Our **prediction** tells us something about how the robot is moving, but only indirectly, and with some uncertainty or inaccuracy.

But if we use all the information available to us, can we get a better answer than **either estimate would give us by itself?** Of course the answer is yes, and that’s what a Kalman filter is for.


# How a Kalman filter sees our problem

Let's look at at the landscape we're trying to interpret. We'll continue with a simple state having only position and velocity.

$$\overrightarrow{x}=\begin{bmatrix}
p\\ 
v
\end{bmatrix}$$

We don’t know what the actual position and velocity are; there are a whole range of possible combinations of position and velocity that might be true, but some of them are more likely than others:

![](http://www.bzarg.com/wp-content/uploads/2015/08/gauss_0-287x300.png)


*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*


The Kalman filter assumes that both variables(position and velocity, in our case) are random and **Gaussian distributed**. Each variable has a **mean** value $\mu$, which is the center of the random distribution (and its most likely state), and a **variance $\sigma^2$**, which is the uncertainty:

![](http://www.bzarg.com/wp-content/uploads/2015/08/gauss_1-300x267.png)

*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*

In the above picture, position and velocity are **uncorrelated**, which means that the state of one variable tells you nothing about what the other might be.

The example below shows something more interesting: Position and velocity are **correlated**. The likelihood of observing a particular position depends on what velocity you have:

![](http://www.bzarg.com/wp-content/uploads/2015/08/gauss_3-300x276.png)

*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*

This kind of situation might arise if, for example, we are estimating a new position based on an old one. If our velocity was high, we probably moved farther, so our position will be more distant. If we’re moving slowly, we didn’t get as far.


This kind of relationship is really important to keep track of, because it gives us **more information:** One measurement tells us something about what the others could be. And that’s the goal of the Kalman filter, we want to squeeze as much information from our uncertain measurements as we possibly can!

This correlation is captured by something called a [**covariance matrix**](https://en.wikipedia.org/wiki/Covariance_matrix). In short, each element of the matrix $\sum_{ij}$ is the degree of correlation between the ith state variable and the jth state variable. (You might be able to guess that the covariance matrix is symmetric, which means that it doesn’t matter if you swap i and j). Covariance matrices are often labelled “$\sum$”, so we call their elements “$\sum_{ij}$”.

![](http://www.bzarg.com/wp-content/uploads/2015/08/gauss_2-300x276.png)

*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*



# Describing the problem with matrices

We need two pieces of information at time **$k$**: we'll call our best estimate **$\hat{x_k}$** ( the mean, usually named **$\mu$**) , and its covariance matrix **$P_k$**.

$$\\hat{x_k} = \begin{bmatrix}
position\\ 
velocity
\end{bmatrix}$$


$$P_k = \begin{bmatrix}
\sum_{pp} & \sum_{pv} \\
\sum_{vp} 
 & \sum_{vv}
\end{bmatrix}$$


Note: It's useful to remember taht the state can contain any number of variables, and represent anything we want.


Next, we need some way to look at the **current state**( at time **k-1**) and **predict the next state** at time **k**. Remember, we don't know which state is the *real* one, but our prediction function doesn't care. It just works on all of them, and gives us a new distribution:

![](http://www.bzarg.com/wp-content/uploads/2015/08/gauss_7-300x276.jpg)


*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*


We can represent this state change step with a matrix, **$F_k$**:

![](http://www.bzarg.com/wp-content/uploads/2015/08/gauss_8-300x276.jpg)

*Image courtesy: [Bzarg](http://www.bzarg.com/p/how-a-kalman-filter-works-in-pictures/#mathybits)*



It takes every point in our original estimate and moves it to a new predicted location, which is where the system would move if that original estimate was the right one.

Let’s apply this. How would we use a matrix to predict the position and velocity at the next moment in the future? We’ll use a really basic kinematic formula:


$$\begin{split} 
\color{deeppink}{p_k} &= \color{royalblue}{p_{k-1}} + \Delta t &\color{royalblue}{v_{k-1}} \\ 
\color{deeppink}{v_k} &= &\color{royalblue}{v_{k-1}} 
\end{split}$$



$$\begin{align}
\color{deeppink}{\mathbf{\hat{x}}_k} &= \begin{bmatrix} 
1 & \Delta t \\ 
0 & 1 
\end{bmatrix}\color{royalblue}{\mathbf{\hat{x}}_{k-1}} \\
&= \mathbf{F}_k \color{royalblue}{{\mathbf{\hat{x}}_{k-1}}}
\end{align}(1)$$

We now have a **prediction matrix** which gives us our next state, but we still don't know how to update the covariance matrix.

If we multiply every point in a distribution by a matrix $\color{firebrick}{\mathbf{A}}$, then this is what happens to its covariance matrix $\Sigma$

$$\begin{equation} 
\begin{split} 
Cov(x) &= \Sigma\\ 
Cov(\color{firebrick}{\mathbf{A}}x) &= \color{firebrick}{\mathbf{A}} \Sigma \color{firebrick}{\mathbf{A}}^T 
\end{split} 
\end{equation}    (2)$$

So combine the last two equations equation(1) and (2), we get

$$\begin{equation} 
\begin{split} 
\color{deeppink}{\mathbf{\hat{x}}_k} &= \mathbf{F}_k \color{royalblue}{\mathbf{\hat{x}}_{k-1}} \\ 
\color{deeppink}{\mathbf{P}_k} &= \mathbf{F_k} \color{royalblue}{\mathbf{P}_{k-1}} \mathbf{F}_k^T 
\end{split} 
\end{equation}$$